In [2]:
import subprocess
import json

# Global variables to track processing state and store strings
process_file = False
original_string = "stream"
ffmpeg_command = " "

def check_media_file(file_path):
    # Initial ffmpeg check
    if initial_ffmpeg_check(file_path) == "Success":
        result = full_workflow(file_path)


def initial_ffmpeg_check(file_path):
    ffmpeg_command = f'ffmpeg -v error -i "{file_path}" -f null -'
    try:
        result = subprocess.run(ffmpeg_command, shell=True, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
        if result.stdout or result.stderr:
            #print (file_path + " file is invalid")          
            return "Failure"
        else:
            #print (file_path + " file is valid") 
            return "Success"
    except subprocess.CalledProcessError as e:
        #print(result.stdout)
        return "Error"


def full_workflow(file_path):
    ffprobe_command = f'ffprobe -loglevel quiet -show_entries format:stream=index,stream,codec_type,codec_name,channel_layout,format=nb_streams -of json "{file_path}"'
    
    try:
        result = subprocess.run(ffprobe_command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stream_info = json.loads(result.stdout)
        streams_count = stream_info['format']['nb_streams']
        for i in range(streams_count):
            process_stream(stream_info['streams'][i], i)

        ffmpeg_string = 'ffmpeg -hide_banner -loglevel 16 -stats -stats_period 10 -i "' + file_path + '"' + ffmpeg_command + ' "' + file_path + '"'
        print ("original_string is: " + original_string)
        print ("ffmpeg_command is: " + ffmpeg_string)
        print ("process_file is:" + str(process_file))
    except subprocess.CalledProcessError as e:
        print(result.stdout)

def process_stream(stream, index):
    global original_string
    codec_type = stream.get('codec_type')
    #print ("codec_type is: " + codec_type)
    codec_name = stream.get('codec_name')
    channel_layout = stream.get('channel_layout')

    original_string = original_string + " 0:" + str(index) + " " + codec_type + ":" + codec_name
    
    if codec_type == 'video':
        process_video_stream(codec_name, index)
    elif codec_type == 'audio':
        process_audio_stream(codec_name, channel_layout, index)
    elif codec_type == 'subtitle':
        process_subtitle_stream(codec_name, index)
    elif codec_type == 'attachment':
        process_attachment_stream(codec_name, index)
    else:
        print(f"Other stream type: codec_name={codec_name}, codec_type={codec_type}")

def process_video_stream(codec_name, index):
    global process_file, ffmpeg_command
    #print ("codec_name is: " + codec_name)
    if codec_name == 'av1':
        #print(f"Video stream {index}: codec_name=av1")
        ffmpeg_command = ffmpeg_command + ' -map 0:' + str(index) + ' -c:v stuff'
        #print (process_file)
        #print (ffmpeg_command)
    else:
        #print(f"Video stream {index}: codec_name={codec_name}")
        ffmpeg_command = ffmpeg_command + ' -map 0:' + str(index) + ' -c:v copy'
        process_file = True
        #print (process_file)
        #print (ffmpeg_command)

def process_audio_stream(codec_name, channel_layout, index):
    global process_file, ffmpeg_command
    #print(f"Audio stream: codec_name={codec_name}, channel_layout={channel_layout}")
    ffmpeg_command = ffmpeg_command + ' -map 0:' + str(index) + ' -c:a copy'
    
def process_subtitle_stream(codec_name, index):
    global process_file, ffmpeg_command
    #print(f"Subtitle stream: codec_name={codec_name}")
    ffmpeg_command = ffmpeg_command + ' -map 0:' + str(index) + ' -c:s copy'

def process_attachment_stream(codec_name, index):
    global process_file, ffmpeg_command
    #print(f"Attachment stream: codec_name={codec_name}")
    ffmpeg_command = ffmpeg_command + ' -map 0:' + str(index) + ' -c:t copy'


hhelo


In [31]:
import subprocess
import json













In [156]:
import subprocess, json, os


def check_video_stream(encoding_decision, i, stream_info, ffmpeg_command):
    # Checks the video stream from check_codecs to determine if the stream needs encoding
    codec_name = stream_info['streams'][i]['codec_name'] 
    desired_video_codec = 'av1'
    print('Steam ' + str(i) + ' codec is: ' + codec_name)
    if codec_name == desired_video_codec:
        ffmpeg_command = ffmpeg_command + ' -map 0:' + str(i) + ' -c:v copy'
    elif codec_name == 'mjpeg':
        ffmpeg_command = ffmpeg_command + ' -map 0:' + str(i) + ' -c:v copy'
    elif codec_name != desired_video_codec: 
        encoding_decision = True
        svt_av1_string = "libsvtav1 -crf 25 -preset 4 -g 240 -pix_fmt yuv420p10le -svtav1-params filmgrain=20:film-grain-denoise=0:tune=0:enable-qm=1:qm-min=0:qm-max=15"
        ffmpeg_command = ffmpeg_command + ' -map 0:' + str(i) + ' -c:v ' + svt_av1_string
    else:
        print ('ignoring for now')
    return encoding_decision, ffmpeg_command

def check_audio_stream(encoding_decision, i, stream_info, ffmpeg_command):
    # Checks the audio stream from check_codecs to determine if the stream needs encoding
    codec_name = stream_info['streams'][i]['codec_name'] 
    # This will be populated at a later date
    #desired_audio_codec = 'aac'
    #if codec_name != desired_video_codec:
    #    encoding_decision = True
    print('Steam ' + str(i) + ' codec is: ' + codec_name)
    ffmpeg_command = ffmpeg_command + ' -map 0:' + str(i) + ' -c:a copy'
    return encoding_decision, ffmpeg_command
    
def check_subtitle_stream(encoding_decision, i, stream_info, ffmpeg_command):
    # Checks the subtitle stream from check_codecs to determine if the stream needs encoding
    codec_name = stream_info['streams'][i]['codec_name'] 
    # This will be populated at a later date
    #desired_subtitle_codec = 'srt'
    #if codec_name != desired_subtitle_codec:
    #    encoding_decision = True
    print('Steam ' + str(i) + ' codec is: ' + codec_name)
    ffmpeg_command = ffmpeg_command + ' -map 0:' + str(i) + ' -c:s copy'
    return encoding_decision, ffmpeg_command

def check_attachmeent_stream(encoding_decision, i, stream_info, ffmpeg_command):
    # Checks the attachment stream from check_codecs to determine if the stream needs encoding
    codec_name = stream_info['streams'][i]['codec_name'] 
    # This will be populated at a later date
    #desired_attachment_codec = '???'
    #if codec_name != desired_attachment_codec:
    #    encoding_decision = True
    print('Steam ' + str(i) + ' codec is: ' + codec_name)
    ffmpeg_command = ffmpeg_command + ' -map 0:' + str(i) + ' -c:t copy'
    return encoding_decision, ffmpeg_command

def check_codecs(stream_info, encoding_decision):
    # Loops through the streams in stream_info from requires_encoding, then
    # calls functions to determine if the steam needs encoding based on stream type conditions 
    streams_count = stream_info['format']['nb_streams']
    ffmpeg_command = str()
    print ('There are : ' + str(streams_count) + ' streams')
    for i in range (0,streams_count):
        codec_type = stream_info['streams'][i]['codec_type'] 
        if codec_type == 'video':
            encoding_decision, ffmpeg_command = check_video_stream(encoding_decision, i, stream_info, ffmpeg_command)
        elif codec_type == 'audio':
            encoding_decision, ffmpeg_command = check_audio_stream(encoding_decision, i, stream_info, ffmpeg_command)
        elif codec_type == 'subtitle':
            encoding_decision, ffmpeg_command = check_subtitle_stream(encoding_decision, i, stream_info, ffmpeg_command)
        elif codec_type == 'attachment':
            encoding_decision, ffmpeg_command = check_attachmeent_stream(encoding_decision, i, stream_info, ffmpeg_command)        
    return encoding_decision, ffmpeg_command


def check_container_extension(file, encoding_decision):
    base, ext = os.path.splitext(file)
    if ext.lower() != '.mkv':
        # Change the extension to .mkv
        file = base + '.mkv'
        encoding_decision = True
    ffmepg_output_file = '/boil_hold/' + file
    return encoding_decision, ffmepg_output_file

def check_container_type(stream_info, encoding_decision, file):
    # Desired container is MKV so we check for that, and pass True for all other container types
    format_name = stream_info['format'].get('format_name')
    print ('format is: ' + format_name)
    if format_name != 'matroska,webm':
        encoding_decision = True
    file, encoding_decision = check_container_extension(file, encoding_decision)
    print ('>>>check_container_type<<<  Container is: ' + format_name + ' so, encoding_decision is: ' + str(encoding_decision))
    return encoding_decision, file
    

def ffprobe_function(file_path):
    # Subprocess call to ffprobe to retrieve video info in JSON format
    ffprobe_command = f'ffprobe -loglevel quiet -show_entries format:stream=index,stream,codec_type,codec_name,channel_layout,format=nb_streams -of json "{file_path}"'
    result = subprocess.run(ffprobe_command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stream_info = json.loads(result.stdout)
    return stream_info


def requires_encoding(file_located_data):
    stream_info = ffprobe_function(file_located_data['file_path'])
    encoding_decision = False
    encoding_decision, ffmepg_output_file_name = check_container_type(stream_info, encoding_decision, file_located_data['file'])
    encoding_decision, ffmpeg_command = check_codecs(stream_info, encoding_decision)
    if encoding_decision == True:
        print ('file needs encoding')
        
    else:
        print ('file does not need encoding')
    print (encoding_decision)
    print (ffmpeg_command)


In [157]:
file_located_data = {
   "directory": "C:\\Users\\cwest\\Videos\\BoilestDev",
   "extension": ".mkv",
   "file": "test22.mkv",
   "file_path": "C:\\Users\\cwest\\Videos\\BoilestDev\\test2.mkv",
   "root": "C:\\Users\\cwest\\Videos\\BoilestDev"
}

requires_encoding(file_located_data)

format is: matroska,webm
>>>check_container_type<<<  Container is: matroska,webm so, encoding_decision is: /boil_hold/test22.mkv
There are : 10 streams
Steam 0 codec is: hevc
Steam 1 codec is: aac
Steam 2 codec is: ass
Steam 3 codec is: ttf
Steam 4 codec is: ttf
Steam 5 codec is: ttf
Steam 6 codec is: ttf
Steam 7 codec is: ttf
Steam 8 codec is: ttf
Steam 9 codec is: ttf
file does not need encoding
True
 -map 0:0 -c:v libsvtav1 -crf 25 -preset 4 -g 240 -pix_fmt yuv420p10le -svtav1-params filmgrain=20:film-grain-denoise=0:tune=0:enable-qm=1:qm-min=0:qm-max=15 -map 0:1 -c:a copy -map 0:2 -c:s copy -map 0:3 -c:t copy -map 0:4 -c:t copy -map 0:5 -c:t copy -map 0:6 -c:t copy -map 0:7 -c:t copy -map 0:8 -c:t copy -map 0:9 -c:t copy


In [153]:
import logging, json, os

def locate_files(arg):
    directories = ['C:\\Users\\cwest\\Videos\\BoilestDev']
    extensions = ['.mp4', '.mkv', '.avi']

    print(f'Searching directories: {directories}')
    print(f'File extensions: {extensions}')

    for file_located in find_files(directories, extensions):
        print('File located, sending to ffprobe function')
        try:
            file_located_data = json.loads(file_located)
            print(json.dumps(file_located_data, indent=3, sort_keys=True))
            #ffprober.delay(file_located_data)  # Uncomment this line to send the task to ffprober
        except json.JSONDecodeError as e:
            logging.error(f'Failed to decode JSON: {e}')
            continue

def find_files(directories, extensions):
    for directory in directories:
        logging.info ('Scanning: ' + directory)
        for root, dirs, files in os.walk(directory):
            for file in files:
                for ext in extensions:
                    if file.lower().endswith(ext.lower()):
                        file_path = os.path.join(root, file)
                        result_dict = {
                            'directory': directory,
                            'root': root,
                            'file': file,
                            'file_path': file_path,
                            'extension': ext
                        }
                        yield json.dumps(result_dict)


In [ ]:
locate_files('fart')